In [3]:
import os
import re
from collections import defaultdict
from tqdm.notebook import tqdm




import pandas as pd

STOCKS_PATH = '../stock-market-dataset/stocks/'

stocks_idx = pd.read_csv('../stock-market-dataset/stocks_idx.csv').dropna(subset="Sector").set_index("Sector").sort_index().reset_index()

In [4]:
sector_map = dict(zip(
    STOCKS_PATH + stocks_idx['Symbol'] + '.csv', 
    stocks_idx['Sector']                           
))

super_dataframes = defaultdict(lambda: pd.DataFrame())

In [5]:
pbar = tqdm(list(sector_map.keys()), desc="Processing Stocks", leave=True)

for stock_file in pbar:
    
    sector = sector_map[stock_file]
    
    pbar.set_postfix(Stock=os.path.basename(stock_file), Sector=sector)                         
    # Read the data file efficiently
    df = pd.read_csv(stock_file, index_col="Date", parse_dates=True)

    # Retrieve the cumulative DataFrame for this class
    sector = re.sub(r'[\s\-]+', '_', sector).lower()
    class_df = super_dataframes[sector]
        
    # 3. Aggregation
    if class_df.empty:
        super_dataframes[sector] = df
    else:
        super_dataframes[sector] = class_df.add(df, fill_value=0)

Processing Stocks:   0%|          | 0/3617 [00:00<?, ?it/s]

In [6]:
for sector, df in super_dataframes.items():
    df = df.iloc[:-1, :]
    df.to_csv(f'./{re.sub(r'[\s\-]+', '_', sector).lower()}.csv')